## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
%%capture
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import string
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv'
data = pd.read_csv(url, sep='\t', usecols=[0, 1])
display(data.head())

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


**Уберем знаки препинания**

In [ ]:
def remove_punctuation(sentence):
  for ch in string.punctuation:
    sentence = sentence.replace(ch,"")
  return sentence
data['review'] = data.apply(lambda row: remove_punctuation(row['review']), axis=1)
display(data.head())

,review,sentiment
0,качество плохое пошив ужасный горловина напере...,negative
1,Товар отдали другому человеку я не получила по...,negative
2,Ужасная синтетика Тонкая ничего общего с предс...,negative
3,товар не пришел продавец продлил защиту без мо...,negative
4,Кофточка голая синтетика носить не возможно,negative


**Приведение к нижнему регистру**

In [ ]:
data['review'] = data.apply(lambda row: row['review'].lower(), axis=1)
display(data.head())

,review,sentiment
0,качество плохое пошив ужасный горловина напере...,negative
1,товар отдали другому человеку я не получила по...,negative
2,ужасная синтетика тонкая ничего общего с предс...,negative
3,товар не пришел продавец продлил защиту без мо...,negative
4,кофточка голая синтетика носить не возможно,negative


**Токенизация**


In [ ]:
from nltk.tokenize import word_tokenize
data['review'] = data.apply(lambda row: word_tokenize(row['review'], language='russian'), axis=1)
display(data.head())

,review,sentiment
0,"[качество, плохое, пошив, ужасный, горловина, ...",negative
1,"[товар, отдали, другому, человеку, я, не, полу...",negative
2,"[ужасная, синтетика, тонкая, ничего, общего, с...",negative
3,"[товар, не, пришел, продавец, продлил, защиту,...",negative
4,"[кофточка, голая, синтетика, носить, не, возмо...",negative


**Удаление стоп слов**

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.remove('не')
def remove_stop_words(sentence):
  return [w for w in sentence if not w in stop_words]
    
data['review'] = data.apply(lambda row: remove_stop_words(row['review']), axis=1)
display(data.head())

,review,sentiment
0,"[качество, плохое, пошив, ужасный, горловина, ...",negative
1,"[товар, отдали, другому, человеку, не, получил...",negative
2,"[ужасная, синтетика, тонкая, общего, представл...",negative
3,"[товар, не, пришел, продавец, продлил, защиту,...",negative
4,"[кофточка, голая, синтетика, носить, не, возмо...",negative


**Лемматизация**

In [ ]:
%%capture
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

In [ ]:
# проверка
morph = MorphAnalyzer()
print(morph.parse('мыши')[0].normal_form)

мышь


In [ ]:
def lemmatize(sentence):
  return [morph.parse(w)[0].normal_form for w in sentence]
data['review'] = data.apply(lambda row: lemmatize(row['review']), axis=1)
## удаление стоп слов еще раз
data['review'] = data.apply(lambda row: remove_stop_words(row['review']), axis=1)
display(data.head())

,review,sentiment
0,"[качество, плохой, пошив, ужасный, горловина, ...",negative
1,"[товар, отдать, человек, не, получить, посылка...",negative
2,"[ужасный, синтетик, тонкий, общий, представить...",negative
3,"[товар, не, прийти, продавец, продлить, защита...",negative
4,"[кофточка, голый, синтетик, носить, не, возможно]",negative


In [ ]:
def convert_to_string(sentence):
  return ' '.join(sentence)
data['review'] = data.apply(lambda row: convert_to_string(row['review']), axis=1)
display(data.head())

,review,sentiment
0,качество плохой пошив ужасный горловина напере...,negative
1,товар отдать человек не получить посылка ладны...,negative
2,ужасный синтетик тонкий общий представить карт...,negative
3,товар не прийти продавец продлить защита согла...,negative
4,кофточка голый синтетик носить не возможно,negative


**Векторизация**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов"
from sklearn.feature_extraction.text import TfidfVectorizer # tf idf векторайзер 
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)


###**Мешок слов (Count vectorizer)**

In [ ]:
accuracy = 0
cv_best_params = 0
# мешок слов
for ngram_ in [(1,2), (1,3), (1,4), (1,5), (2,2), (2, 3), (2, 4), (2,5), (3,3), (3,4), (3,5), (4, 4), (4,5), (5,5)]:
  model = Pipeline([('cv', CountVectorizer(ngram_range = ngram_)), ('classifier',MultinomialNB())])
  model.fit(x_train, y_train)
  cv_y_predicted = model.predict(x_test)
  cur_accuracy = accuracy_score(cv_y_predicted, y_test)
  if cur_accuracy > accuracy:
    cv_best_params = ngram_
    accuracy = cur_accuracy
print("best accuracy: ", accuracy)
print("best params: ", cv_best_params)

model = Pipeline([('cv', CountVectorizer(ngram_range = cv_best_params)), ('classifier',MultinomialNB())])
model.fit(x_train, y_train)
count_vectorizer_y_predicted = model.predict(x_test)

best accuracy:  0.7244176141624384
best params:  (1, 5)


##**tf-idf TfIdfVectorizer**

In [ ]:
# tf-idf
accuracy = 0
tf_idf_best_params = {}
for ngram_ in [(1,2), (1,3), (1,4), (1,5), (2,2), (2, 3)]:
  for min_df in [0, 0.01]:
    for max_df in [0.1,0.5, 0.7,]:
      for max_features in [None, 1_000, 10_000]:
        model = Pipeline([('tf_idf', TfidfVectorizer(ngram_range = ngram_, min_df=min_df, max_df=max_df, max_features=max_features)),
                          ('classifier',MultinomialNB())])
        model.fit(x_train, y_train)
        tf_idf_y_predicted = model.predict(x_test)
        cur_accuracy = accuracy_score(tf_idf_y_predicted, y_test)
        if cur_accuracy > accuracy:
          tf_idf_best_params = {'ngram_': ngram_, 'min_df': min_df, 'max_df': max_df, 'max_features': max_features}
          accuracy = cur_accuracy
print("best accuracy: ", accuracy)
print("best params: ", tf_idf_best_params)

model = Pipeline([('tf_idf', TfidfVectorizer(ngram_range = tf_idf_best_params['ngram_'], min_df=tf_idf_best_params['min_df'],
                  max_df=tf_idf_best_params['max_df'], max_features=tf_idf_best_params['max_features'])),('classifier',MultinomialNB())])
model.fit(x_train, y_train)
tf_idf_y_predicted = model.predict(x_test)

best accuracy:  0.7248250064812415
best params:  {'ngram_': (1, 3), 'min_df': 0, 'max_df': 0.5, 'max_features': None}


##**Символьные n-граммы**
нижней границей для ngram_range будет 3 символа так как наборы из 2 и 1 символа точно не представляют смыловую нагрузку \
а в качестве верхней границы можно взять среднее количество символов в  униграммах и биграммах

In [ ]:
# поиск средней длины  униграм и биграм
from nltk import ngrams
def unigram_len_mean_in_review(sentence):
  return np.mean([len(str(w[0])) for w in list(ngrams(sentence.split(), 1))]) if sentence!='' else 0

unigram_means = data.apply(lambda row: unigram_len_mean_in_review(row['review']), axis=1)
print(np.mean(unigram_means))

def bigram_len_mean_in_review(sentence):
  return np.mean([len(str(w[0]+w[1])) for w in list(ngrams(sentence.split(), 2))]) if sentence!='' else 0

bigram_means = data.apply(lambda row: bigram_len_mean_in_review(row['review']), axis=1)
print(np.mean(bigram_means))

6.422474172108056


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


12.765217341372786


In [ ]:
# char ngrams
model = Pipeline([('char_ngrams', CountVectorizer(analyzer='char', ngram_range = (3, 7))),
                  ('classifier',MultinomialNB())])
model.fit(x_train, y_train)
char_ngrams_y_predicted = model.predict(x_test)
print(accuracy_score(char_ngrams_y_predicted, y_test))

model = Pipeline([('char_ngrams', CountVectorizer(analyzer='char', ngram_range = (3, 14))),
                  ('classifier',MultinomialNB())])
model.fit(x_train, y_train)
char_ngrams_y_predicted = model.predict(x_test)
print(accuracy_score(char_ngrams_y_predicted, y_test))

0.7128254509092256
0.7258990407762675


###**Сравнение моделей по метрикам качества**

In [ ]:
from sklearn.metrics import  precision_score, recall_score, f1_score, accuracy_score
def report(y_predicted, y_test):
  presicion = precision_score(y_predicted, y_test, average='macro')
  recall = recall_score(y_predicted, y_test, average='macro')
  f1 = f1_score(y_predicted, y_test, average='macro')
  accuracy = accuracy_score(y_predicted, y_test)
  return (presicion, recall, f1, accuracy)

count_vectorizer = [v for v in report(count_vectorizer_y_predicted, y_test)]
tf_idf = [v for v in report(tf_idf_y_predicted, y_test)]
ngrams = [ v for v in report(char_ngrams_y_predicted, y_test)]
data_ = {"count_vectorizer": count_vectorizer, 'tf_idf':tf_idf, 'ngrams': ngrams}
report_ = pd.DataFrame(data_, index=['presicion','recall','f1-score', 'accuracy']).T
display(report_)

,presicion,recall,f1-score,accuracy
count_vectorizer,0.724378,0.728062,0.725953,0.724418
tf_idf,0.724943,0.730436,0.726429,0.724825
ngrams,0.725993,0.731291,0.727614,0.725899


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx? \
так как совпадения abca abcx пересекаются 

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
sentence = "some string with a lot of words"
sentence = re.split(' ', sentence)
result = [re.findall('^\w\w', w) for w in sentence]
result = [w[0] for w in result if w]
print(result)

['so', 'st', 'wi', 'lo', 'of', 'wo']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
sentence = 'мама не мыла раму. а может и мыла. а может и нет . а кто его знает. Никто не уверен'
result = re.split('\.', sentence, maxsplit=2)
print(result)

['мама не мыла раму', ' а может и мыла', ' а может и нет . а кто его знает. Никто не уверен']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
sentence = 'sdklfngsdlag8w8 htui34u uitht230 tp4h2tu3 j4j'
result = re.sub('[0-9]','DIG', sentence )
print(result)

sdklfngsdlagDIGwDIG htuiDIGDIGu uithtDIGDIGDIG tpDIGhDIGtuDIG jDIGj


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
sentence = 'http://colab.research.google.com/drive/1Y2L90BskeWB9AEHmHyHiPJjm6PTc7IjF#scrollTo=KwNS9zt4WhAv some info https://www.youtube.com/playlist?list=PL4SWw1noBwCQxh-dk3WK3eYDH-ZOcMsWR'
result = re.sub('https?://\\S+', '', sentence)
print(result)

 some info 


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('(?<!\w)\w{4,}(?!\w)')
prog.findall('344 sdfs  skkfs kkk khhhsdkfg kiiw4 kkk3 23 4 424  jtgn3j iii')

['sdfs', 'skkfs', 'khhhsdkfg', 'kiiw4', 'kkk3', 'jtgn3j']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
sentence = ("abc.test@gmail.com", "xyz@test.in", "test.first@analyticsvidhya.com", "first.test@rest.biz")
prog = re.compile('@[\w\.-]+')
print(list(prog.findall(s) for s in sentence))

[['@gmail.com'], ['@test.in'], ['@analyticsvidhya.com'], ['@rest.biz']]
